In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import autoreject
import csv
import matplotlib.pyplot as plt
import pandas as pd
import mne
import os
import sys
#sys.path.append('../..')
from atpbar import atpbar
#from utils.csv_utils_new import *
#from utils.lsl_utils_new import *
#from utils.MNE_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TESTING HAND CLEANING

# testing cleaning strategies

In [77]:
def plot_evokeds (epochs):
    evokeds = dict()
    subset = epochs['1']
    evokeds['standard'] = list(subset.iter_evoked())
    subset = epochs['-1']
    evokeds['oddball'] = list(subset.iter_evoked())
    mne.viz.plot_compare_evokeds(evokeds, picks='Fz', ci=0.95, title=f'global oddball evoked response: subject {str(subject)}, {condition}') #Fz for MMN, Pz for LTP

In [187]:
subject = 7 #0-9, not 6
for condition in ['jhana', 'mindfulness']:
    fname = f'/Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/{condition}_sub{str(subject)}-epo.fif'

    epochs = mne.read_epochs(fname)
    
#    ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,n_jobs=1, verbose=True)
#    ar.fit(epochs)
#    epochs, reject_log = ar.transform(epochs, return_log=True)

    #epochs.filter(1, 40)
    #epochs = epochs.set_eeg_reference(ref_channels='average')

    evokeds = dict()
    subset = epochs['1']
    evokeds['standard'] = list(subset.iter_evoked())
    subset = epochs['-1']
    evokeds['oddball'] = list(subset.iter_evoked())
    #mne.viz.plot_compare_evokeds(evokeds, picks='Fz', ci=0.95, title=f'global oddball evoked response: subject {str(subject)}, {condition}') #Fz for MMN, Pz for LTP
    mne.viz.plot_compare_evokeds(evokeds, picks='Fz', ci=0.95, title=f'global oddball evoked response: {condition}') #Fz for MMN, Pz for LTP


Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/jhana_sub7-epo.fif ...
    Found the data of interest:
        t =    -101.56 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1157 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/mindfulness_sub7-epo.fif ...
    Found the data of interest:
        t =    -101.56 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
995 matching events found
No baseline correction applied
0 projection items activated


2023-10-04 10:34:54.165 python[39098:5274541] +[CATransaction synchronize] called within transaction
2023-10-04 10:35:08.836 python[39098:5274541] +[CATransaction synchronize] called within transaction


In [ ]:
# clean EPOCHS

def clean_epochs_full (epochs):
    ica = mne.preprocessing.ICA(n_components=32, random_state=97, max_iter="auto")
    ica.fit(epochs)
    muscle_idx_auto, scores = ica.find_bads_muscle(epochs)
    print (f"excluded ICA components: {muscle_idx_auto}")
    epochs_ica = ica.apply(epochs.copy(), exclude=muscle_idx_auto)
    
    ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,n_jobs=1, verbose=False)
    ar.fit(epochs_ica)
    epochs_ar, reject_log = ar.transform(epochs_ica, return_log=True)
    epochs_ar.filter(None, 40)
    print (f"retention rate: {len(epochs_ar)}/{len(epochs)}")
    return (epochs, epochs_ica, epochs_ar, reject_log)

# plot ALL evokeds (Jhana & Mindfulness)

In [4]:
epochs_path = '/Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/'


for meditation in ['jhana', 'mindfulness']:
    epochs = mne.read_epochs(f'{epochs_path}{meditation}_all-epo.fif')
    
    epochs.crop(tmax=0.4)
    evokeds = dict()
    subset = epochs['1']
    evokeds['standard'] = list(subset.iter_evoked())
    subset = epochs['-1']
    evokeds['oddball'] = list(subset.iter_evoked())

    mne.viz.plot_compare_evokeds(evokeds, picks='Fz', ci=0.95, title=f'global oddball evoked response during {meditation}', truncate_yaxis=True) #Fz for MMN, Pz for LTP
    
"""
    #the code below computes the mean value for all evokeds
    mean = []
    start = 231 #in samples
    end = 384 
    for condition in ['standard', 'oddball']:
        for i in range(len(evokeds[condition])):
            tmp = np.mean(evokeds[condition][i].pick(['Fz']).data[0,start:end])
            mean.append(tmp)
        print (f'for {meditation} and {condition}, mean value: {np.mean(mean)}')
        
"""

Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/jhana_all-epo.fif ...
    Found the data of interest:
        t =    -101.56 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
909 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/mindfulness_all-epo.fif ...
    Found the data of interest:
        t =    -101.56 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
638 matching events found
No baseline correction applied
0 projection items activated


"\n    #the code below computes the mean value for all evokeds\n    mean = []\n    start = 231 #in samples\n    end = 384 \n    for condition in ['standard', 'oddball']:\n        for i in range(len(evokeds[condition])):\n            tmp = np.mean(evokeds[condition][i].pick(['Fz']).data[0,start:end])\n            mean.append(tmp)\n        print (f'for {meditation} and {condition}, mean value: {np.mean(mean)}')\n        \n"

# compare evokeds evokeds INDIVIDUAL (Jhana & Mindfulness)

In [5]:
export_path = '/Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/'
subjects = ['sub0', 'sub1', 'sub2', 'sub3', 'sub4', 'sub5', 'sub6', 'sub7', 'sub8', 'sub9']
subjects = ['sub0']
days = ['day1', 'day2', 'day3', 'day4']
days = ['day1']
meditations = ['jhana', 'mindfulness']

# Initialize an empty list to store each DataFrame
df = []
dfs = []

for subject in subjects:
    for day in days:
        for meditation in meditations:            
            file_path = f'{export_path}{subject}-{meditation}_mmn-{day}-epo.fif'
            try: 
                epochs = mne.read_epochs(file_path)                
                evokeds = dict()
                subset = epochs['1']
                evokeds['standard'] = list(subset.iter_evoked())
                subset = epochs['-1']
                evokeds['oddball'] = list(subset.iter_evoked())

                mean_by_condition = []
                for cond_name in ['oddball', 'standard']:
                    mean_10_15 = []
                    mean_15_20 = []
                    for i in range(len(evokeds[cond_name])):
                        start = np.where(evokeds[cond_name][i].times >= 0.1)[0][0]  # Find the index where time is 0s
                        end = np.where(evokeds[cond_name][i].times <= 0.15)[0][-1]  # Find the index where time is 1s
                        tmp = np.mean(evokeds[cond_name][i].pick(['Fz']).data[0,start:end])
                        mean_10_15.append(tmp)
                        
                        start = np.where(evokeds[cond_name][i].times >= 0.15)[0][0]  # Find the index where time is 0s
                        end = np.where(evokeds[cond_name][i].times <= 0.2)[0][-1]  # Find the index where time is 1s
                        tmp = np.mean(evokeds[cond_name][i].pick(['Fz']).data[0,start:end])
                        mean_15_20.append(tmp)
                        
                    mean_10_15 = np.mean(mean_10_15)
                    mean_15_20 = np.mean(mean_15_20)
                    df = pd.DataFrame([[mean_10_15]], columns=['mean_10_15'])
                    df['mean_15_20'] = mean_15_20
                    df['subject'] = subject
                    df['day'] = day
                    df['condition'] = cond_name
                    df['meditation'] = meditation
                    dfs.append(df)
            except: 
                print ("========================")
                print ('this file did not work:')
                print (file_path)
                print ("$$$$$$$$$$$$$$$$$$$$$$$$")


# Combine all DataFrames
df = pd.concat(dfs, ignore_index=True)

print('\n' * 2)
mean_comparison = df.groupby(['meditation', 'condition'])['mean_10_15'].mean()
print(mean_comparison)
print('\n' * 2)
mean_comparison = df.groupby(['meditation', 'condition'])['mean_15_20'].mean()
print(mean_comparison)

Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub0-jhana_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
369 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub0-mindfulness_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
173 matching events found
No baseline correction applied
0 projection items activated



meditation   condition
jhana        oddball     -0.000004
             standard    -0.000002
mindfulness  oddball     -0.000005
             standard    -0.000004
Name: mean_10_15, dtype: float64



meditation   condition
jhana        oddball     -0.000009
             standard    -0.000001
mindfulness  oddball     -0.000006
            

## stats tetst

In [3]:
from scipy.stats import ttest_rel

def clean_series_pairs(series1, series2):
    """Return cleaned series by removing pairs with NaN values in either series."""
    mask = ~np.isnan(series1) & ~np.isnan(series2)
    return series1[mask], series2[mask]

def paired_ttest_all_sessions (df_input, meditation1, meditation2, condition1, condition2, feature, avg_subject=False):
    if avg_subject: 
        df = df_input.groupby(['subject', 'meditation', 'condition'])[feature].mean().reset_index()
    else:
        df=df_input
    series1 = df[(df['meditation'] == meditation1) & (df['condition'] == condition1)][feature]
    series2 = df[(df['meditation'] == meditation2) & (df['condition'] == condition2)][feature]
    series1_cleaned, series2_cleaned = clean_series_pairs(np.array(series1), np.array(series2))
    t_stat, p_val = ttest_rel(series1_cleaned, series2_cleaned)
    print (f'results for {feature}: {meditation1}–{condition1} VS. {meditation2}–{condition2}')
    print(f"paired t-test result: t = {t_stat}, p = {p_val}")

In [88]:
paired_ttest_all_sessions(df, 'jhana', 'jhana', 'oddball', 'standard', 'mean_15_20')
paired_ttest_all_sessions(df, 'mindfulness', 'mindfulness', 'oddball', 'standard', 'mean_15_20')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'oddball', 'oddball', 'mean_15_20')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'standard', 'standard', 'mean_15_20')

paired_ttest_all_sessions(df, 'jhana', 'jhana', 'oddball', 'standard', 'mean_10_15')
paired_ttest_all_sessions(df, 'mindfulness', 'mindfulness', 'oddball', 'standard', 'mean_10_15')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'oddball', 'oddball', 'mean_10_15')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'standard', 'standard', 'mean_10_15')

results for mean_15_20: jhana–oddball VS. jhana–standard
paired t-test result: t = -3.2750933242124107, p = 0.002485548886068595
results for mean_15_20: mindfulness–oddball VS. mindfulness–standard
paired t-test result: t = -2.4605106745015712, p = 0.01911021246758858
results for mean_15_20: jhana–oddball VS. mindfulness–oddball
paired t-test result: t = -1.3454955387771332, p = 0.18792242991215172
results for mean_15_20: jhana–standard VS. mindfulness–standard
paired t-test result: t = 0.1973519743357277, p = 0.8446937737442358
results for mean_10_15: jhana–oddball VS. jhana–standard
paired t-test result: t = -1.6904136483862924, p = 0.10037509418011176
results for mean_10_15: mindfulness–oddball VS. mindfulness–standard
paired t-test result: t = -0.33670723563581734, p = 0.738407142707858
results for mean_10_15: jhana–oddball VS. mindfulness–oddball
paired t-test result: t = -1.7628186474824896, p = 0.08748046411692244
results for mean_10_15: jhana–standard VS. mindfulness–standard
p

## testing features for Fpz

In [5]:
export_path = '/Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/'
subjects = ['sub0', 'sub1', 'sub2', 'sub3', 'sub4', 'sub5', 'sub6', 'sub7', 'sub8', 'sub9']
days = ['day1', 'day2', 'day3', 'day4']
meditations = ['jhana', 'mindfulness']

# Initialize an empty list to store each DataFrame
df = []
dfs = []

for subject in subjects:
    for day in days:
        for meditation in meditations:            
            file_path = f'{export_path}{subject}-{meditation}_mmn-{day}-epo.fif'
            try: 
                epochs = mne.read_epochs(file_path)                
                evokeds = dict()
                subset = epochs['1']
                evokeds['standard'] = list(subset.iter_evoked())
                subset = epochs['-1']
                evokeds['oddball'] = list(subset.iter_evoked())

                mean_by_condition = []
                for cond_name in ['oddball', 'standard']:
                    mean_50_70 = []
                    mean_40_50 = []
                    for i in range(len(evokeds[cond_name])):
                        start = np.where(evokeds[cond_name][i].times >= 0.5)[0][0]  # Find the index where time is 0s
                        end = np.where(evokeds[cond_name][i].times <= 0.7)[0][-1]  # Find the index where time is 1s
                        tmp = np.mean(evokeds[cond_name][i].pick_channels(['Fpz']).data[0,start:end])
                        mean_50_70.append(tmp)
                        
                        start = np.where(evokeds[cond_name][i].times >= 0.4)[0][0]  # Find the index where time is 0s
                        end = np.where(evokeds[cond_name][i].times <= 0.5)[0][-1]  # Find the index where time is 1s
                        tmp = np.mean(evokeds[cond_name][i].pick_channels(['Fpz']).data[0,start:end])
                        mean_40_50.append(tmp)
                        
                    mean_50_70 = np.mean(mean_50_70)
                    mean_40_50 = np.mean(mean_40_50)
                    df = pd.DataFrame([[mean_50_70]], columns=['mean_50_70'])
                    df['mean_40_50'] = mean_40_50
                    df['subject'] = subject
                    df['day'] = day
                    df['condition'] = cond_name
                    df['meditation'] = meditation
                    dfs.append(df)
            except: 
                print ("========================")
                print ('this file did not work:')
                print (file_path)
                print ("$$$$$$$$$$$$$$$$$$$$$$$$")


# Combine all DataFrames
df = pd.concat(dfs, ignore_index=True)

print('\n' * 2)
mean_comparison = df.groupby(['meditation', 'condition'])['mean_10_15'].mean()
print(mean_comparison)
print('\n' * 2)
mean_comparison = df.groupby(['meditation', 'condition'])['mean_15_20'].mean()
print(mean_comparison)

Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub0-jhana_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
99 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub0-mindfulness_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
65 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub0-jhana_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
233 matching events found
No baseline cor

        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
142 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub3-mindfulness_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
67 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub3-jhana_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated


/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub3-mindfulness_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
361 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub3-jhana_mmn-day3-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub3-mindfulness_mmn-day3-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
192 matching events found
No baseli

/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub5-jhana_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
344 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub5-mindfulness_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
360 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub5-jhana_mmn-day3-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
161 matching events found
No baseline c

    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
61 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub8-mindfulness_mmn-day2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
229 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub8-jhana_mmn-day3-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis

/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/bitbrain/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub8-mindfulness_mmn-day4-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
266 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub9-jhana_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
78 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jonasmago/PhD_code_data/Jhana_US/analysis/epochs_mmn/sub9-mindfulness_mmn-day1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
340 matching events found
No basel

KeyError: 'Column not found: mean_10_15'

In [6]:
paired_ttest_all_sessions(df, 'jhana', 'jhana', 'oddball', 'standard', 'mean_50_70')
paired_ttest_all_sessions(df, 'mindfulness', 'mindfulness', 'oddball', 'standard', 'mean_50_70')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'oddball', 'oddball', 'mean_50_70')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'standard', 'standard', 'mean_50_70')

paired_ttest_all_sessions(df, 'jhana', 'jhana', 'oddball', 'standard', 'mean_40_50')
paired_ttest_all_sessions(df, 'mindfulness', 'mindfulness', 'oddball', 'standard', 'mean_40_50')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'oddball', 'oddball', 'mean_40_50')
paired_ttest_all_sessions(df, 'jhana', 'mindfulness', 'standard', 'standard', 'mean_40_50')

results for mean_50_70: jhana–oddball VS. jhana–standard
paired t-test result: t = -1.823701869420401, p = 0.07726543575009233
results for mean_50_70: mindfulness–oddball VS. mindfulness–standard
paired t-test result: t = -2.928057418839717, p = 0.006047301614155763
results for mean_50_70: jhana–oddball VS. mindfulness–oddball
paired t-test result: t = 0.5520880566672495, p = 0.584723631262074
results for mean_50_70: jhana–standard VS. mindfulness–standard
paired t-test result: t = 1.4371110186439655, p = 0.15956671585254983
results for mean_40_50: jhana–oddball VS. jhana–standard
paired t-test result: t = -1.7213839384528926, p = 0.09454489114930884
results for mean_40_50: mindfulness–oddball VS. mindfulness–standard
paired t-test result: t = -2.6282890122027647, p = 0.012786830657581374
results for mean_40_50: jhana–oddball VS. mindfulness–oddball
paired t-test result: t = 0.14114149168666407, p = 0.8886429498957689
results for mean_40_50: jhana–standard VS. mindfulness–standard
pair